In [ ]:
import pandas as pd
import numpy as np
import time
import csv
#import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_finance import candlestick2_ochl,volume_overlay
import talib
from math import log, exp
from matplotlib import dates as mdates
from matplotlib import ticker as mticker
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY,YEARLY
from matplotlib.dates import MonthLocator,MONTHLY
import datetime as dt
import pylab
import h5py
from PIL import Image
import os
import threading
import multiprocessing as mp
from multiprocessing import Pool
from random import shuffle
from numba import vectorize, float64, cuda
import imageio
import glob
import cv2
  
hdf5_path = 'pic_sum_new.h5' 

%matplotlib inline



In [ ]:
stocknum = {'0051','1102','1216','1227','1314','1319','1434','1451','1476','1477','1504','1536','1560','1590',
            '1605','1704','1717','1718','1722','1723','1789','1802','1909','2015','2049','2059','2106','2201',
            '2204','2207','2227','2231','2312','2313','2324','2327','2337','2344','2347','2352','2353','2356',
            '2360','2371','2376','2377','2379','2385','2439','2448','2449','2451','2478','2492','2498','2542',
            '2603','2606','2610','2615','2618','2723','2809','2812','2834','2845','2867','2888','2912','2915',
            '3019','3034','3044','3051','3189','3231','3406','3443','3532','3673','3682','3702','3706','4137',
            '4915','4943','4958','5264','5522','5871','6005','6116','6176','6239','6269','6285','6409','6414',
            '6415','6452','6456','8454','8464','9910','9914','9917','9921','9933','9938','9941','9945'}

# 將資料傳入 

將圖片改成RGB維度

In [ ]:
pic_sum = []
table_sum=[]

for stockid in stocknum:
    df = pd.read_hdf(stockid+'_table_sumchange.h5','stock_data_table')
    pic_addr = glob.glob('D:/program/stock_pic/' +stockid + 'pic/'+'*.jpg')
    
    for item in pic_addr:
        item = item.replace('\\','/')
        #print (item)
        pic_sum.append(item)
    for table in df.values:
        table_sum.append(table)
    print(len(pic_sum))
    print(len(table_sum))

    
c = list(zip(pic_sum,table_sum))
shuffle(c)
addrs, labels = zip(*c)
addrs = list(addrs)
labels = list(labels)

train_addrs = addrs[0:int(0.6*len(addrs))]
train_labels = labels[0:int(0.6*len(labels))]


val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]

test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]
    
#train_shape = (len(train_addrs), 380, 383, 3)
#val_shape = (len(val_addrs), 380, 383, 3)
#test_shape = (len(test_addrs), 380, 383, 3)


In [ ]:
train_addrs

In [ ]:
hdf5_file = h5py.File(hdf5_path, mode='w')
for i,a in enumerate(train_addrs) :
    train_addrs[i] =  a.encode('utf8')
for i,a in enumerate(val_addrs) :
    val_addrs[i] =  a.encode('utf8')
for i,a in enumerate(test_addrs) :
    test_addrs[i] =  a.encode('utf8')

hdf5_file.create_dataset("train_img", data = train_addrs)
hdf5_file.create_dataset("val_img", data=val_addrs)
hdf5_file.create_dataset("test_img", data=test_addrs)


hdf5_file.create_dataset("train_labels", (len(train_addrs),4), np.int8)
hdf5_file["train_labels"][...] = train_labels

hdf5_file.create_dataset("val_labels", (len(val_addrs),4), np.int8)
hdf5_file["val_labels"][...] = val_labels

hdf5_file.create_dataset("test_labels", (len(test_addrs),4), np.int8)
hdf5_file["test_labels"][...] = test_labels


In [ ]:
hdf5_file["train_img"][0]